<div style="border: 1px solid #CCCCFF; border-radius: 1px;">
<div style="margin: 5px">
<b class="alert-heading">Комментарий ревьюера</b>
<p>Борис, привет!</p>
<p>Меня зовут Алексей Секоцкий. Поздравляю с подготовкой очередного проекта. Предлагаю обращаться друг к другу на «ты» если нет возражений. Ниже стандартный блок с условными обозначениями:</p>
<div class="alert alert-success" role="alert">
всё отлично
</div>
<div class="alert alert-warning" role="alert">
рекомендации на будущее (не требующие доработки проекта)
</div>
<div class="alert alert-danger" role="alert">
критичные моменты требующие внимания (доработки)
</div>
<div class="alert alert-info">
информационные комментарии и итоговый
</div>

<hr>
<p>Пожалуйста, не изменяй и не удаляй мои комментарии – они потребуются для повторной проверки (при необходимости). Задавай вопросы и описывай сделанные изменения, помечая их любым удобным наглядным способом.</p>
    
   
<div class="alert alert-info">
<b class="alert-heading">Комментарий студента:</b>
Добрый день, Алексей, Спасибо за проверку проекта! У меня есть одна просьба: одолжить один цвет для моих комментариев исправлений ошибок и тд, так как другие цвета у меня не получается подгрузить: 
</div>
<div class="alert alert-primary" role="alert">
  Это основное уведомление — check it out!
</div>
<div class="alert alert-secondary" role="alert">
  Это дополнительное уведомление — check it out!
</div>
</div>
<div class="alert alert-light" role="alert">
  Это инфо-уведомление — check it out!
</div>
<div class="alert alert-dark" role="alert">
  Это темное уведомление — check it out!
</div>
    
<div class="alert alert-info">
<b class="alert-heading">Комментарий студента:</b>
У меня возник вопрос по комментарию: "Использование кросс-валидации в обычном виде нежелательно, потому что в валидационную выборку попадают части частот слов и происходит утечка." 
Что нужно указать, чтобы избежать данной утечки?  


<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Введение" data-toc-modified-id="Введение-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Введение<a id="intro"></a></a></span></li><li><span><a href="#Оглавление:" data-toc-modified-id="Оглавление:-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Оглавление:</a></span></li><li><span><a href="#1.-Подготовка" data-toc-modified-id="1.-Подготовка-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>1. Подготовка<a id="preparing"></a></a></span><ul class="toc-item"><li><span><a href="#1.1-Открытие-и-изучиние-файлов" data-toc-modified-id="1.1-Открытие-и-изучиние-файлов-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>1.1 Открытие и изучиние файлов<a id="open"></a></a></span></li><li><span><a href="#1.2-Подготовка-данных-к-обучению" data-toc-modified-id="1.2-Подготовка-данных-к-обучению-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>1.2 Подготовка данных к обучению<a id="preparation"></a></a></span><ul class="toc-item"><li><span><a href="#Разбиение-датасета-на-тестовые-и-тренировочные-выборки." data-toc-modified-id="Разбиение-датасета-на-тестовые-и-тренировочные-выборки.-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Разбиение датасета на тестовые и тренировочные выборки.</a></span></li><li><span><a href="#Вычисление-TF-IDF" data-toc-modified-id="Вычисление-TF-IDF-3.2.2"><span class="toc-item-num">3.2.2&nbsp;&nbsp;</span>Вычисление TF-IDF</a></span></li></ul></li></ul></li><li><span><a href="#2.-Обучение-" data-toc-modified-id="2.-Обучение--4"><span class="toc-item-num">4&nbsp;&nbsp;</span>2. Обучение <a id="training"></a></a></span><ul class="toc-item"><li><span><a href="#2.1-Логистическая-регрессия-без-кросс-валидации" data-toc-modified-id="2.1-Логистическая-регрессия-без-кросс-валидации-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>2.1 Логистическая регрессия без кросс-валидации<a id="3.1"></a></a></span></li><li><span><a href="#2.2-Логистическая-регрессия-с-кросс-валидацией" data-toc-modified-id="2.2-Логистическая-регрессия-с-кросс-валидацией-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>2.2 Логистическая регрессия с кросс-валидацией<a id="3.2"></a></a></span></li><li><span><a href="#2.3-Случайный-лес" data-toc-modified-id="2.3-Случайный-лес-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>2.3 Случайный лес<a id="3.3"></a></a></span></li><li><span><a href="#2.4-Дерево-решений" data-toc-modified-id="2.4-Дерево-решений-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>2.4 Дерево решений<a id="3.4"></a></a></span></li></ul></li><li><span><a href="#Вывод-" data-toc-modified-id="Вывод--5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод <a id="conclusion"></a></a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп» с BERT


## Введение<a id="intro"></a>


Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

План работы: 
1. Загрузка и подготовка данных.
2. Обучение разных моделей. 
3. Вывод.

Описание данных:
 - *text*  - текст комментария, 
 - *toxic* - целевой признак.




## 1. Подготовка<a id="preparing"></a>
### 1.1 Открытие и изучиние файлов<a id="open"></a>

In [1]:
# Импорт библеотек

from IPython.display import display
import pandas as pd
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import fbeta_score, make_scorer
from tqdm import notebook
from sklearn.tree import DecisionTreeClassifier
import re
from lightgbm import LGBMClassifier

In [2]:
# Открытие данных
df = pd.read_csv('toxic_comments.csv')
display(df.head(5))

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
df.info()
df.duplicated().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


0

В загруженном датасете 159571 записей. Пропусков нет, дубликатов нет.

### 1.2 Подготовка данных к обучению<a id="preparation"></a>
#### Очистка данных

In [4]:
def clear_text(text):
    return " ".join(re.sub(r'[^a-zA-Z]',' ', text).split())
df['lemma'] = df['text'].apply(clear_text)

In [5]:
print(df['lemma'].sample(5))

23324     Backstreet Boys Backstreet Boys article was bl...
141663    Image Why can we use these images Image X japa...
88428     Hey Jerk You need to read the editnotice well ...
35802     YA GODDAMIT IDIOT MY IP CHANGES WHEN AN ADMIST...
130301    Thank you for experimenting with Wikipedia You...
Name: lemma, dtype: object


#### Разбиение датасета на тестовые, тренировочные и валидационные выборки.<a id='valid'></a>

In [6]:
x_train, x_test, y_train, y_test = train_test_split(
    df.lemma, df.toxic, test_size=0.4, random_state=12345)
x_valid, x_test, y_valid, y_test = train_test_split(
    x_test, y_test, test_size=0.5, random_state=12345)

print('Размер тренировочной выборки:', x_train.count())
print('Размер валидационной выборки:',x_valid.count())
print('Размер тестовой выборки:',x_test.count())


Размер тренировочной выборки: 95742
Размер валидационной выборки: 31914
Размер тестовой выборки: 31915


#### Вычисление TF-IDF
Для тренировочной выборки

In [7]:
corpus = x_train
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords, lowercase=True)
tf_idf = count_tf_idf.fit_transform(corpus)
tf_idf.shape

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sugan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(95742, 125610)

<div class="alert alert-warning">
<b>Комментарий ревьюера</b>

У нас текст в основном английский. Переводить в Unicode (`.values.astype('U')`) не нужно. В тренажере мы работали с русским текстом, поэтому там это было оправдано. Путем экспериментов коллега вывел, что на полном датасете из-за предобразвания в Unicode отъедатся около 3 Гб памяти и ядро падает :) Нам такое не нужно :)
</div>

<div class="alert alert-danger">
<b>Комментарий ревьюера</b>

При использовании TF-IDF необходимо также добавить очистку или лемматизацию текста. Проще реализовать очистку (в теории по спринту – тема «Регулярные выражения») и привести все символы к нижнему регистру (.lower()  или в tf-idf lowercase=True). Если будет желание сделать лемматизацию, то необходимо обратить внимание на следующие особенности: лемматизатор должен поддерживать английский язык. Также должен либо уметь сразу работать с предложениями, либо, если ты выберешь лемматризатор, который работает по одному слову, то разбить предложения на слова, провести лемматизацию и снова собрать отдельные слова в предложения. <a href="https://webdevblog.ru/podhody-lemmatizacii-s-primerami-v-python/">Здесь</a> можно прочитать подробнее.
<br/>

<div class="alert alert-info">
<b class="alert-heading">Комментарий студента:</b>

1. Убрал перевод в Unicode.

2. Добавил очистку и привел к нижнему регистру в tf-idf lowercase=True
    
3. Пытался добавить лемматизацию текста через NLTK и spaCy, но библиотека NLTK отказывалось лемматизировать список, а через цикл каждое слово приводить к лемме мучительно долго выходит, а библиотека spaCy постоянно требовало что-либо подгрузить дополнительно, но к jupiter notebook у меня не получилось жто всё подключить.
</div>

Для валидационной выборки

In [8]:
corpus = x_valid
tf_idf_valid = count_tf_idf.transform(corpus)
tf_idf_valid.shape

(31914, 125610)

Для тестовой выборки

In [9]:
corpus = x_test
tf_idf_test = count_tf_idf.transform(corpus)
tf_idf_test.shape

(31915, 125610)

<div class="alert alert-success">
<b>Комментарий ревьюера</b>

Хорошо, что TF-IDF настраивается на обучающей выборке, а для теста используется transform без fit.
</div>

## 2. Обучение <a id='training'></a>
### 2.1 Логистическая регрессия<a id="3.1"></a>

In [10]:
model = LogisticRegression(random_state=12345, class_weight='balanced').fit(tf_idf, y_train)
predictions = model.predict(tf_idf_valid)
f1 = f1_score(y_valid, predictions)
print('f1 для логистической регрессии без кросс-валидации:', f1)

# Список показателей f1 моделей
f1_list = []
f1_list.append(f1.round(2))

f1 для логистической регрессии без кросс-валидации: 0.7532967032967033


D:\Program Files\anaconda3\envs\praktikum\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


<div class="alert alert-danger">
<b>Комментарий ревьюера</b>

Использование кросс-валидации в обычном виде нежелательно, потому что в валидационную выборку попадают части частот слов и происходит утечка. Если будет принято решение отказаться от кросс-валидации, то необходимо поделить данные выше на 3 выборки: добавить валидационную.
<br/>

<div class="alert alert-info">
<b class="alert-heading">Комментарий студента:</b>

1. Убрал кросс-валидацию

2. Добавил [валидационную выборку](#valid)

  

### 2.2 Случайный лес<a id="3.3"></a>

In [11]:
best_model = None
best_f1 = 0


for depth in notebook.tqdm(range(50, 150, 10)):
    model = RandomForestClassifier(max_depth=depth, random_state=12345, n_estimators=1).fit(tf_idf, y_train)
    preditions = model.predict(tf_idf_valid)
    f1 = f1_score(y_valid,preditions)
    
    if best_f1 < f1:
        best_model = model
        best_f1 = f1
        best_depth = depth
        
            
print('Налилучший показатель F1 = {} для случайного леса достигается при глубине равной {}:'.format(best_f1,best_depth) )

# запись f1
f1_list.append(best_f1.round(2))

  0%|          | 0/10 [00:00<?, ?it/s]

Налилучший показатель F1 = 0.45306532663316584 для случайного леса достигается при глубине равной 130:


### 2.3 Дерево решений<a id="3.4"></a>

In [12]:
best_depth = 0
best_model = None
best_f1 = 0

for depth in notebook.tqdm(range(50, 150, 10)):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth).fit(tf_idf, y_train)
    predictions = model.predict(tf_idf_valid)
    f1 = f1_score(y_valid, predictions)
    if best_f1 < f1:
        best_f1 = f1
        best_depth = depth
        best_model = model

print('Налилучший показатель F1 = {} для дерева решений достигается при глубине равной {}:'.format(best_f1,best_depth) )

# запись f1
f1_list.append(best_f1.round(2))

  0%|          | 0/10 [00:00<?, ?it/s]

Налилучший показатель F1 = 0.7340479836651354 для дерева решений достигается при глубине равной 100:


### 2.4 LGBMClassifier

In [13]:
best_model = None
best_f1 = 0
best_est = 0
for est in notebook.tqdm(range(100, 151, 25)):
    model = LGBMClassifier(random_state=12345, n_estimators=est).fit(tf_idf, y_train)
    preditions = model.predict(tf_idf_valid)
    f1 = f1_score(y_valid, preditions)
    if f1 > best_f1:
        best_model = model
        best_f1 = f1
        best_est = est
            
        
print('Налилучший показатель F1 = {} для дерева решений достигается при количестве деревьев равным {}:'.format(best_f1,best_est) )
# запись f1
f1_list.append(best_f1.round(2))

  0%|          | 0/3 [00:00<?, ?it/s]

Налилучший показатель F1 = 0.763264586638463 для дерева решений достигается при количестве деревьев равным 150:


In [14]:
index_list = ['Логистическая регрессия без кросс-валидации', 'Дерево решений', "Случайный лес", 'LGBMClassifier' ]
d = {'f1': f1_list}
result = pd.DataFrame(data=d, index = index_list)
display(result)

,f1
Логистическая регрессия без кросс-валидации,0.75
Дерево решений,0.45
Случайный лес,0.73
LGBMClassifier,0.76


Для тестирования возьмем Логистическую регрессию и LGBMClassifier, так как показатели F1 больше 0.75.

<div class="alert alert-warning">
<b>Комментарий ревьюера</b>

Неплохой результат в этом проекте может дать LGBMClassifier, но перевод в Unicode необходимо отключить иначе с большой вероятностью не хватить ресурсов и ядро «упадет».
</div>

<div class="alert alert-info">
<b class="alert-heading">Комментарий студента:</b>

1. Добавил LGBMClassifier

## 3. Тестирование
### 3.1 Логистическая регрессия

In [15]:
model = LogisticRegression(random_state=12345, class_weight='balanced').fit(tf_idf, y_train)
predictions = model.predict(tf_idf_test)
f1 = f1_score(y_test, predictions)
print('f1 для логистической регрессии:', f1)

f1 для логистической регрессии: 0.7524587893059981


D:\Program Files\anaconda3\envs\praktikum\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


### 3.2 LGBMClassifier

In [16]:
model = LGBMClassifier(random_state=12345, n_estimators=500).fit(tf_idf, y_train)
preditions = model.predict(tf_idf_test)
f1 = f1_score(y_test, preditions)
print('f1 для LGBMClassifier:', f1)

f1 для LGBMClassifier: 0.7743170349747696


## Вывод <a id='conclusion'></a>

Проведена предобработка данных и обучена модель линейной регрессией, случайным лесом, деревом решиний.
Наилучшие показатели метрики показала модель, обученная LGBMClassifier: F1 = 0.77.

Исходя из  результатов теста предлагаю модель LGBMClassifier, так как удовлетворяет условию поставленной задаче.


<div class="alert alert-success">
<b>Комментарий ревьюера</b>

Отлично, требуемая метрика достигнута.
</div>

<div class="alert alert-info">
<b>Итоговый комментарий ревьюера</b>

Хорошее начало. Есть несколько важных моментов, требующих внимания: добавить очистку или лемматизацию и разобраться с кросс-валидацией. Буду ждать новую версию!
<hr>
</div>

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны